# 畳み込みニューラルネットワーク

## 全体像
<div style="text-align:center">
    <img src="img/cnn/cnn10.png" width="800">
    <img src="img/cnn/cnn8.png" width="800">
</div>

## 畳み込み層

<div style="text-align:center">
    <img src="img/cnn/cnn1.png" width="500">
    <img src="img/cnn/cnn2.png" width="500">
    <img src="img/cnn/cnn13.png" width="1000">
    <img src="img/cnn/cnn14.png" width="600">
</div>


## 畳み込み層: フィルタ
<div style="text-align:center">
<img src="img/cnn/cnn12.png" width="300">
<img src="img/cnn/cnn5.png" width="500">
<img src="img/cnn/cnn6.png" width="500">
</div>

### プーリング層
<div style="text-align:center">
    <img src="img/cnn/cnn11.png" width="300">    
    <img src="img/cnn/cnn9.png" width="300">
</div>

### Feature Map
<div>
    <img src="img/cnn/cnn4.png" width="400">
</div>

### その他

<div>
<img src="img/cnn/cnn3.png" width="300">
<img src="img/cnn/cnn7.png" width="300">
<img src="img/cnn/cnn15.png" width="300">
<div>


式: 

$$
【畳み込み層 forward】\\
a_{i,j,m} = \sum_{k=0}^{K-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1}x_{(i+s),(j+t),k}w_{s,t,k,m}+b_{m}\\
$$

In [1]:
# ゼロつく
def forward(self, x):
    w_n, w_c, w_h, w_w = self.w.shape   # (n_samples, n_channels, height, width)
    n, c, h, w = x.shape                # (n_samples, n_channels, height, width)
    out_h = 1 + int((h + 2*self.pad - w_h) / self.stride)
    out_w = 1 + int((w + 2*self.pad - w_w) / self.stride)

    col = im2col(x, w_h, w_w, self.stride, self.pad)
    col_w = self.w.reshape(w_n, -1).T

    out = np.dot(col, col_w) + self.b
    out = out.reshape(n, out_h, out_w, -1).transpose(0, 3, 1, 2)

    return out

# jop
def forward(self, X):
    N,C,H,W = self.X.shape
    F,C,FH,FW = self.W.shape
    OH,OW = self.output_shape2d(H,W,self.P,self.P,FH,FW,self.S,self.S)
    A = np.zeros([N,F,OH,OW])
    self.X_pad = np.pad(X,((0,0),(0,0),(self.P,self.P),(self.P,self.P)))

    for n in range(N): # n_samples
        for ch in range(F): # Output channels
            for row in range(0,H,self.S): # Vertical slide
                for col in range(0,W,self.S): # # Horizontal Slide
                    tmp = self.X_pad[n,:,row:row+FH,col:col+FW] * self.W[ch,:,:,:]
                    A[n,ch,row,col] = np.sum(tmp) + self.B[ch]

    return self.activation.forward(A)

$$
【畳み込み層 backward】\\
\frac{\partial L}{\partial w_{s,t,k,m}} = \sum_{i=0}^{N_{out,h}-1}\sum_{j=0}^{N_{out,w}-1} \frac{\partial L}{\partial a_{i,j,m}}x_{(i+s)(j+t),k}\\
\frac{\partial L}{\partial b_{m}} = \sum_{i=0}^{N_{out,h}-1}\sum_{j=0}^{N_{out,w}-1}\frac{\partial L}{\partial a_{i,j,m}}
$$

In [ ]:
# ゼロつく
def backward(self, dout):
    fn, c, fh, fw = self.w.shape
    dout = dout.transpose(0,2,3,1).reshape(-1, fn)

    self.d_b = np.sum(dout, axis=0)
    self.d_w = np.dot(self.col.T, dout)
    self.d_w = self.d_w.transpose(1, 0).reshape(fn, c, fh, fw)

    dcol = np.dot(dout, self.col_w.T)
    dx = col2im(dcol, self.x.shape, fh, fw, self.stride, self.pad)

    return dx


# jop
def backward(self, dZ):
    """
    backward
    Parameters
    ----------
    dA : ndarray of the following form, shape (batch_size, n_nodes2)
        The gradient flowed in from behind.
    Returns
    ----------
    dZ : ndarray of the following form, shape (batch_size, n_nodes1)
        forward slope
    """

    dA = self.activation.backward(dZ)
    N,C,H,W,F,FH,FW,OH,OW = self.params

    dZ = np.zeros(self.X_pad.shape)
    self.dW = np.zeros(self.W.shape)
    self.dB = np.zeros(self.B.shape)

    # dZ
    # Batch
    for n in range(N):
        # Output channels
        for ch in range(F):
            # Vertical slide
            for row in range(0,H,self.S):
                # Horizontal Slide
                for col in range(0,W,self.S):
                    dZ[n,:,row:row+FH,col:col+FW] += dA[n,ch,row,col]*self.W[ch,:,:,:]

    dl_rows = range(self.P),range(H+self.P,H+2*self.P,1)
    dl_cols = range(self.P),range(W+self.P,W+2*self.P,1)

    dZ = np.delete(dZ,dl_rows,axis=2)
    dZ = np.delete(dZ,dl_cols,axis=3)

    # dW
    # Batch
    for n in range(N):
        # Output channels
        for ch in range(F):
            # Vertical slide
            for row in range(OH):
                # Horizontal Slide
                for col in range(OW):
                    self.dW[ch,:,:,:] += dA[n,ch,row,col]*self.X_pad[n,:,row:row+FH,col:col+FW]

    # dB
    # Output channels
    for ch in range(F):
        self.dB[ch] = np.sum(dA[:,ch,:,:])

    # Update
    self = self.optimizer.update(self)

    return dZ

$$
【前に流す誤差の数式】\\
\frac{\partial L}{\partial x_{i,j,k}} = \sum_{m=0}^{M-1}\sum_{s=0}^{F_{h}-1}\sum_{t=0}^{F_{w}-1} \frac{\partial L}{\partial a_{(i-s),(j-t),m}}w_{s,t,k,m}
$$